In [3]:
from theano.sandbox import cuda
cuda.use('gpu2')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(2), GPU number 0 is already in use.


In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


# Setup

In [5]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = np.expand_dims(x_train, 1)
x_test = np.expand_dims(x_test, 1)

In [7]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [8]:
mean_x = x_train.mean().astype(np.float32)
std_x = x_train.std().astype(np.float32)

In [9]:
# method to normalise the input 
def normalise(x): return (x-mean_x)/std_x

# Linear Model

In [10]:
x_train.shape

(60000, 1, 28, 28)

In [11]:
def get_lin_model():
    model = Sequential([
             Lambda(normalise, input_shape=x_train.shape[1:]),
             Flatten(),
             Dense(10, activation='softmax')
    ])
    model.compile(Adam(), 'categorical_crossentropy', ['accuracy'])
    return model

In [12]:
lm = get_lin_model()

In [13]:
#Generate batches from image data
gen = image.ImageDataGenerator()
batches = gen.flow(x_train, y_train, batch_size=64)
val_batches = gen.flow(x_test, y_test, batch_size=64)

In [14]:
#Run the model for 1 epoch using the default learning rate
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4285 - acc: 0.8737 - val_loss: 0.3020 - val_acc: 0.9136


In [15]:
lm.optimizer.lr = 0.1

In [16]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.2988 - acc: 0.9146 - val_loss: 0.2997 - val_acc: 0.9168


In [17]:
lm.optimizer.lr = 0.01

In [18]:
lm.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.2847 - acc: 0.9206 - val_loss: 0.2845 - val_acc: 0.9216
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.2772 - acc: 0.9229 - val_loss: 0.2800 - val_acc: 0.9202
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.2724 - acc: 0.9244 - val_loss: 0.2820 - val_acc: 0.9214
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.2683 - acc: 0.9255 - val_loss: 0.2696 - val_acc: 0.9251


# Single Dense Layer

In [32]:
def get_fc_model():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Flatten(),
            Dense(512, activation='softmax'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
fc = get_fc_model()

In [34]:
fc.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 1.5455 - acc: 0.8841 - val_loss: 1.0126 - val_acc: 0.9206


In [35]:
fc.optimizer.lr=0.1

In [36]:
fc.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 0.7477 - acc: 0.9240 - val_loss: 0.5532 - val_acc: 0.9278


In [37]:
fc.optimizer.lr=0.01

In [38]:
fc.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.4626 - acc: 0.9297 - val_loss: 0.3962 - val_acc: 0.9319
Epoch 2/4
60000/60000 [==============================] - 13s - loss: 0.3513 - acc: 0.9355 - val_loss: 0.3270 - val_acc: 0.9359
Epoch 3/4
60000/60000 [==============================] - 13s - loss: 0.3021 - acc: 0.9382 - val_loss: 0.3106 - val_acc: 0.9319
Epoch 4/4
60000/60000 [==============================] - 13s - loss: 0.2740 - acc: 0.9409 - val_loss: 0.2832 - val_acc: 0.9367


# Basic VGG-style CNN

In [44]:
def get_model():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Convolution2D(32, 3, 3, activation='relu'),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            Convolution2D(64, 3, 3, activation='relu'),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [45]:
model = get_model()

In [46]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1085 - acc: 0.9667 - val_loss: 0.0524 - val_acc: 0.9828


In [47]:
model.optimizer.lr=0.1

In [48]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.0374 - acc: 0.9883 - val_loss: 0.0223 - val_acc: 0.9932


In [49]:
model.optimizer.lr=0.01

In [50]:
model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/8
60000/60000 [==============================] - 20s - loss: 0.0238 - acc: 0.9922 - val_loss: 0.0262 - val_acc: 0.9918
Epoch 2/8
60000/60000 [==============================] - 20s - loss: 0.0188 - acc: 0.9941 - val_loss: 0.0364 - val_acc: 0.9907
Epoch 3/8
60000/60000 [==============================] - 20s - loss: 0.0149 - acc: 0.9953 - val_loss: 0.0265 - val_acc: 0.9931
Epoch 4/8
60000/60000 [==============================] - 21s - loss: 0.0125 - acc: 0.9960 - val_loss: 0.0365 - val_acc: 0.9907
Epoch 5/8
60000/60000 [==============================] - 20s - loss: 0.0116 - acc: 0.9964 - val_loss: 0.0275 - val_acc: 0.9932
Epoch 6/8
60000/60000 [==============================] - 20s - loss: 0.0077 - acc: 0.9976 - val_loss: 0.0179 - val_acc: 0.9947
Epoch 7/8
60000/60000 [==============================] - 20s - loss: 0.0087 - acc: 0.9972 - val_loss: 0.0292 - val_acc: 0.9927
Epoch 8/8
60000/60000 [==============================] - 20s - loss: 0.0077 - acc: 0.9977 - val_loss: 0.0270 - 

# Reducing Overfitting